In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
import fitz
import os
import boto3
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

F:\anaconda3\envs\python_3_7\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



# Code to Convert PDF to PNG

In [2]:
def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/New/'
        output_file = f"{path+os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [442]:
convert_pdf2img('Makro Invoices/Makro Invoices/Convert/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra.pdf')

Deprecation: 'preRotate' removed from class 'Matrix' after v1.19 - use 'prerotate'.
Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.


## Summary ########################################################
File:Makro Invoices/Makro Invoices/Convert/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra.pdf
Pages:None
Output File(s):['Makro Invoices/Makro Invoices/New/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra_page1.png']
###################################################################


Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


['Makro Invoices/Makro Invoices/New/60064_B38245866 DEFAULT B0134 TYPE_20220203_6_FacturaCompra_page1.png']

In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [6]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [448]:
Ima = "02.05.21_MAKRO, S.A._245,58_page1.png"

In [449]:
def find_amounts(text):
    amounts = re.findall(r'\d+\.\d{2}\b', text)
    floats = [float(amount) for amount in amounts]
    unique = list(dict.fromkeys(floats))
    if len(unique)==0:
        return 0
    else:
        return max(unique)

In [450]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
       "Document_Type":"",
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
tot = []
doc_type = []
print(type(text_list))
dat['Total']=find_amounts(text.replace(',','.'))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        

        a = a.replace(',','.')
        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
            print(b)
            
        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
            print(b)
            
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('/',''))
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            di = b[0].replace('-','')
            dit = di.replace(',','')
            dat["Vat Number"] = dit.replace('.','')
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            di = b[0].replace('-','')
            dit = di.replace(',','')
            dat["Vat Number"] = dit.replace('/','')

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
        
        if "ALBARAN" in a or "Albaran" in a:
            dat["Document_Type"] = "Albaran"
            doc_type.append("Albaran")
        elif "Factura" in a or "FACTURA" in a  or "factura" in a :
            dat["Document_Type"] = "Factura"
            doc_type.append("Factura")
        if len(doc_type)>1:
            dat["Document_Type"] = doc_type[0]
        
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{1}/\d{2}/\d{2}",a)):
            b = re.findall("\d{1}/\d{2}/\d{2}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        
        elif (re.search("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)
            dat["Date"] = b[0].replace('.','-')
            date.append(b[0].replace('.','-'))
    
        if len(date)>1:
            dat["Date"] = date[0].replace('/','-')

    
    
       
        
        

<class 'list'>
Wall time: 10.3 s


In [451]:
dat

{'Vat Number': 'A28647451',
 'Total': 224.24,
 'Date': '09-05-2021',
 'Document_Type': 'Factura',
 'Invoice_Number': ''}

In [452]:
print(dat['Date'][0:6])

09-05-


In [421]:
dat
if len(dat['Date'])==10:
    pass
elif len(dat['Date'])==8:
    dat['Date'] = dat['Date'][0:6]+'20'+dat['Date'][6:]

In [422]:
dat

{'Vat Number': '',
 'Total': 575.31,
 'Date': '22-03-2022',
 'Document_Type': 'Factura',
 'Invoice_Number': ''}

In [49]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number  Total        Date Document_Type Invoice_Number
0  B16507295  604.3  24-05-2022                             


In [51]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data['Vat Number']= data['Vat Number'].str.replace('/', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until



,Vat Number,Total,Date,Document_Type,Invoice_Number
0,B16507295,604.3,24-05-2022,,


In [13]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [14]:
df2 = formats.query("last == True")

In [15]:
df2

,formato,nombre,nif,last
8892,\d\d\d\d\d\d\d,Isayca Tenerife SLU,B38988242,True
12519,\d\d\d\d\d\d\d,Isayca Tenerife SLU,B38988242,True


In [16]:
inv = 0
for i in df2['formato']:
    inv = i
if inv ==0:
    pass
else:
    factura = re.compile(inv)
    mo = factura.search(text)
    if mo is None:
        pass
    else:
        dat["Invoice_Number"]=mo.group()

In [17]:
dat

{'Vat Number': 'B38988242',
 'Total': '273.22',
 'Date': '3-02-20',
 'Document_Type': 'Factura',
 'Invoice_Number': '1825768'}

In [453]:
im = Image.open(Ima)

In [454]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [455]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [456]:
len(dataframes)

3

In [457]:
d = dataframes[0]

In [458]:
d.head(20)

,Fecha de ventar,02/05/2021 09:2
0,Fecha impresión:,02.05.2021 09:2
1,Fecha de entrega:,02/05/2021 09:2


In [459]:
d.columns

Index(['Fecha de ventar', '02/05/2021 09:2'], dtype='object')

In [430]:
merged_col = False
print(type(d))
for i in (d.columns):
    if i == '':
        print(i)

        merged_col = True

<class 'pandas.core.frame.DataFrame'>




In [460]:
a= d[d.iloc[:,1].str.contains("Lot")==True]

In [461]:
a

,Fecha de ventar,02/05/2021 09:2


In [462]:
for i in d.iloc[:,0]:
    if 'Lote' in i:
        print('yes')

In [463]:
if "Código Descripción" in d.columns:
    print('HI')

In [464]:
def get_desc(desc):
    return desc.split(" ")[0].strip(" ")

In [465]:
if merged_col:
    print("hi")
    if "Código Descripción" in d.columns:
        d['Codigo'] = d["Código Descripción"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Código Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
    else:
        d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
        
        
#    d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
#    d['Codigo']= d['Codigo'].str.replace('/', '')
#    d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
#    d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
#    d['DESCRIPCIÓN']= d['Código'].str.replace('/', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN'].str.replace('[0-9]', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+d.iloc[:,1]
#    d =d.drop(d.columns[[0, 1]],axis = 1) 
#    first_column = d.pop('Codigo')
#    d.insert(0,'Codigo',first_column)
#    second_column = d.pop('DESCRIPCIÓN')
#    d.insert(1,'DESCRIPCIÓN',second_column)

hi


KeyError: 'Código'

In [466]:
if merged_col:
    this_data = d
    print('Yes')

Yes


In [467]:
str1 ='DESCRIPCIÓN'
str2 = "Descrip. artículo"
str3 = "Artículo/ Cod.cliente"
str4 = 'DESCRIPCIÓN:'
str5 = "CONCEPTO"
str6 = "UDS"
str7 = "Concepto"
str8 = "Descripción"
str9 = "DESCRIPCION"   
str10 = "Description articulo"
str11 = "Description"
str12 = "DESCRIPCIÓN ARTICULO"
str13 = "Unidades Descripción"
str14 = "PRODUCTO"
str15 = "Articulo"
str16 = 'Denominación /Referencia'
str17 = 'DESCRIPCIÓN'
str18 = 'REF.'
str19 = "Codigo"
str20 = "Lote"
str21 = "MM Num. articulo"
str22 = 'Artículo'
zero = False
one = False
two = False
three = False

if not merged_col:
    if "ARTÍCULO" in dataframes[0].columns or "CODI EAN/CÓDIGO EAN" in dataframes[0].columns or "MM Num. artículo" in dataframes[0].columns  or 'CÓDIGO' in dataframes[0].columns or 'MM Num.' in dataframes[0].columns or str3 in dataframes[0].columns or str4 in dataframes[0].columns or str5 in dataframes[0].columns or str6 in dataframes[0].columns or str7 in dataframes[0].columns or str8 in dataframes[0].columns or str9 in dataframes[0].columns or str10 in dataframes[0].columns or str11 in dataframes[0].columns or str12 in dataframes[0].columns or str13 in dataframes[0].columns or str14 in dataframes[0].columns or str15 in dataframes[0].columns or str16 in dataframes[0].columns or str17 in dataframes[0].columns or str18 in dataframes[0].columns or str19 in dataframes[0].columns or str20 in dataframes[0].columns or str21 in dataframes[0].columns or str22 in dataframes[0].columns:
            this_data = pd.DataFrame(dataframes[0])
            print('---------0')
            zero = True
    if len(dataframes)>1: 
        if "ARTÍCULO" in dataframes[1].columns or "CODI EAN/CÓDIGO EAN" in dataframes[1].columns or "MM Num. artículo" in dataframes[1].columns or 'CÓDIGO' in dataframes[1].columns or 'MM Num.' in dataframes[1].columns or str3 in dataframes[1].columns or str4 in dataframes[1].columns or str5 in dataframes[1].columns or str6 in dataframes[1].columns or str7 in dataframes[1].columns or str8 in dataframes[1].columns or str9 in dataframes[1].columns or str10 in dataframes[1].columns or str11 in dataframes[1].columns or str12 in dataframes[1].columns or str13 in dataframes[1].columns or str14 in dataframes[1].columns or str15 in dataframes[1].columns or str16 in dataframes[1].columns or str17 in dataframes[1].columns or str18 in dataframes[1].columns or str19 in dataframes[1].columns or str20 in dataframes[1].columns or str21 in dataframes[1].columns or str22 in dataframes[1].columns:        
                this_data = pd.DataFrame(dataframes[1])
                print('---------1')
                one = True
    if len(dataframes)>2:        
        if "ARTÍCULO" in dataframes[2].columns or "CODI EAN/CÓDIGO EAN" in dataframes[2].columns or "MM Num. artículo" in dataframes[2].columns or 'CÓDIGO' in dataframes[2].columns or 'MM Num.' in dataframes[2].columns or str3 in dataframes[2].columns or str4 in dataframes[2].columns or str5 in dataframes[0].columns or str6 in dataframes[2].columns or str7 in dataframes[2].columns or str8 in dataframes[2].columns or str9 in dataframes[2].columns or str10 in dataframes[2].columns or str11 in dataframes[2].columns or str12 in dataframes[2].columns or str13 in dataframes[2].columns or str14 in dataframes[2].columns or str15 in dataframes[2].columns or str16 in dataframes[2].columns or str16 in dataframes[2].columns or str18 in dataframes[2].columns or str19 in dataframes[2].columns or str20 in dataframes[2].columns or str21 in dataframes[2].columns or str22 in dataframes[2].columns:
                this_data = pd.DataFrame(dataframes[2])
                print('---------2')
                two = True
    if len(dataframes)>3:
        if "ARTÍCULO" in dataframes[3].columns or "CODI EAN/CÓDIGO EAN" in dataframes[3].columns or "MM Num. artículo" in dataframes[3].columns or 'CÓDIGO' in dataframes[3].columns or 'MM Num.' in dataframes[3].columns or str3 in dataframes[3].columns or str4 in dataframes[3].columns or str5 in dataframes[3].columns or str6 in dataframes[3].columns or str7 in dataframes[3].columns or str8 in dataframes[3].columns or str9 in dataframes[3].columns or str10 in dataframes[3].columns or str11 in dataframes[3].columns or str12 in dataframes[3].columns or str13 in dataframes[3].columns or str14 in dataframes[3].columns or str15 in dataframes[3].columns or str16 in dataframes[3].columns or str16 in dataframes[3].columns or str18 in dataframes[3].columns or str19 in dataframes[3].columns or str20 in dataframes[3].columns or  str21 in dataframes[3].columns or str22 in dataframes[3].columns:
            this_data = pd.DataFrame(dataframes[3])
            print('---------3')
            three = True
else:
    pass
    
this_data


,Fecha de ventar,02/05/2021 09:2
0,Fecha impresión:,02.05.2021 09:2
1,Fecha de entrega:,02/05/2021 09:2


In [468]:
below_table = ''
if len(dataframes)>1:
    if zero:
        below_table = dataframes[1]
        print('---------')
if len(dataframes)>2:    
    if one:
        below_table = dataframes[2]
        print('---------')
        
if len(dataframes)>3:
    if two:
        below_table = dataframes[3]
        print('---------')
if len(dataframes)>3:
    if three:
        below_table = dataframes[4]
        print('---------')

In [469]:
below_table

''

In [470]:
if len(this_data)>4:
    try:
        this_data.columns = this_data.columns.str.replace('SELECTED  ', '')
        this_data.columns = this_data.columns.str.replace('SELECTED ', '')
        this_data.columns = this_data.columns.str.replace('SELECTED', '')
        this_data.columns = this_data.columns.str.replace('NOT_', '')
        this_data.columns = this_data.columns.str.replace('NOT_SELECTED  ', '')
        this_data.columns = this_data.columns.str.replace('NOT_SELECTED', '')
        big = this_data.columns[0]
        cont = this_data.columns[2]
        lot = this_data.columns[1]
        prec = this_data.columns[3]
        cant = this_data.columns[-5]
        imp = this_data.columns[-2]
        importe = this_data.columns[-3]
        third = this_data.columns[-4]
        this_data = this_data.loc[(this_data[big] != "Número de pedido") & (this_data[big] != "*** Número de pedido") & (this_data[big]!="*** Número de") & (this_data[big]!="*** Número")]
        this_data = this_data.loc[(this_data[big] != "Entregado a: TABERNA") & (this_data[big] !="Entregado a: TASCA") & (this_data[big]!="Entregado a: TABERNA OSUNA")
              & (this_data[big]!= "Entregado a: TABERNA osuna")& (this_data[big]!="Fin de número") ]
        this_data = this_data.loc[(this_data[imp]!= "Spain Fecha:") & (this_data[imp]!= "Spain Fecha:0") & (this_data[imp]!= "Spain Fecha:1")]
        this_data = this_data.loc[this_data[cont] !="LOT:"]
        this_data = this_data.loc[this_data[cant] !="Total"]
        this_data = this_data.loc[this_data[cant]!="LA LAGUNA" ]
        this_data = this_data.replace('',np.nan)
        this_data = this_data.dropna(axis=0, thresh=3, how="any")
        limitPer = len(this_data) * .100
        this_data = this_data.dropna(thresh=limitPer, axis=1)
        this_data = this_data.replace(np.nan,'0')
    except:
        pass

In [471]:
data = this_data
big = this_data

In [472]:
#for i in this_data.columns:
#    if i == '':
#        this_data.drop(i,  inplace=True)
if len(big.columns)==3:
    print('Hi')

In [473]:
this_data.head()

,Fecha de ventar,02/05/2021 09:2
0,Fecha impresión:,02.05.2021 09:2
1,Fecha de entrega:,02/05/2021 09:2


In [474]:
# for  second column
this_data["second_count"] = this_data.iloc[:, 1].str.count("\D")
this_data["second_length"] = this_data.iloc[:, 1].str.len()
this_data["second_digit_percentage"] = this_data["second_count"] / this_data["second_length"] * 100



In [475]:
# for third column
this_data["third_count"] = this_data.iloc[:, 2].str.count("\D")
this_data["third_length"] = this_data.iloc[:, 2].str.len()
this_data["third_digit_percentage"] = this_data["third_count"] / this_data["third_length"] * 100



AttributeError: Can only use .str accessor with string values!

In [382]:
len(this_data.columns)

12

In [383]:
this_data.head()

,Código,Concepto,Cantidad,Precio unit.,Impuestos,Total,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,LAT.013,"Burrata ""LS"" GR.125 LOTE. cad. / /21",5,"15,00",IVA 4%,"75,00",31,36,86.111111,0,1,0.0
1,POM.001,"Pomodori Pelati Kg.2,5 ""CORRADO""",6,"3,80",IVA 10%,"22,80",30,32,93.750000,0,1,0.0


In [384]:
# for  forth column
dftwo = False
if len(data.columns)==3:
    this_data["forth_count"] = this_data.iloc[:, 3].str.count("\D")
    this_data["forth_length"] = this_data.iloc[:, 3].str.len()
    this_data["forth_digit_percentage"] = this_data["forth_count"] / this_data["forth_length"] * 100
    print('Im True')
    dftwo = True



In [385]:
this_data.fillna(0)

,Código,Concepto,Cantidad,Precio unit.,Impuestos,Total,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,LAT.013,"Burrata ""LS"" GR.125 LOTE. cad. / /21",5,"15,00",IVA 4%,"75,00",31,36,86.111111,0,1,0.0
1,POM.001,"Pomodori Pelati Kg.2,5 ""CORRADO""",6,"3,80",IVA 10%,"22,80",30,32,93.750000,0,1,0.0


In [386]:
df = this_data.query("second_digit_percentage>30.0")
df1 = this_data.query("third_digit_percentage>30.0")
if dftwo:
    df2.fillna(0)
    df2 = this_data.query("forth_digit_percentage>60.0")


In [387]:
df.shape

(2, 12)

In [388]:
#df2 = this_data.query("second_digit_percentage>30.0")
this_data.shape

(2, 12)

In [389]:
this_data.head()

,Código,Concepto,Cantidad,Precio unit.,Impuestos,Total,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,LAT.013,"Burrata ""LS"" GR.125 LOTE. cad. / /21",5,"15,00",IVA 4%,"75,00",31,36,86.111111,0,1,0.0
1,POM.001,"Pomodori Pelati Kg.2,5 ""CORRADO""",6,"3,80",IVA 10%,"22,80",30,32,93.750000,0,1,0.0


In [390]:
desc = ''
if len(df.columns) == len(this_data.columns):
    desc = df.iloc[:,1]
    print(' I got in ')
elif len(df1.columns)== len(this_data.columns):
    desc = df1.iloc[:,2]
    print('I got it ')
elif len(df2.columns)== len(this_data.columns):
    desc = df2.iloc[:,3]
    print('I got it2 ')

 I got in 


In [391]:
this_data.head()

,Código,Concepto,Cantidad,Precio unit.,Impuestos,Total,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,LAT.013,"Burrata ""LS"" GR.125 LOTE. cad. / /21",5,"15,00",IVA 4%,"75,00",31,36,86.111111,0,1,0.0
1,POM.001,"Pomodori Pelati Kg.2,5 ""CORRADO""",6,"3,80",IVA 10%,"22,80",30,32,93.750000,0,1,0.0


In [392]:
dfff = ''
description_list = ["Descripción","Descrip. artículo","DESCRIPCION","Description articulo","Description","DESCRIPCIÓN ARTICULO","Unidades Descripción"]
if len(desc) == 0:
    print('Empty')
    for i in ((description_list)):
        if i in this_data.columns:
            print(i)
            dfff = this_data[i]    
            break
       

In [393]:
desc

0    Burrata "LS" GR.125 LOTE. cad. / /21
1        Pomodori Pelati Kg.2,5 "CORRADO"
Name: Concepto, dtype: object

In [394]:
if len(desc)!=0:
    desc_data = pd.DataFrame(desc)
    desc_data.head()
elif len(dfff)!=0:
    desc_data = pd.DataFrame(dfff)
    desc_data.head()

In [395]:
desc_data.head()

,Concepto
0,"Burrata ""LS"" GR.125 LOTE. cad. / /21"
1,"Pomodori Pelati Kg.2,5 ""CORRADO"""


In [396]:
this_data

,Código,Concepto,Cantidad,Precio unit.,Impuestos,Total,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,LAT.013,"Burrata ""LS"" GR.125 LOTE. cad. / /21",5,"15,00",IVA 4%,"75,00",31,36,86.111111,0,1,0.0
1,POM.001,"Pomodori Pelati Kg.2,5 ""CORRADO""",6,"3,80",IVA 10%,"22,80",30,32,93.750000,0,1,0.0


In [397]:
len(data.columns)

12

In [398]:
this_data

,Código,Concepto,Cantidad,Precio unit.,Impuestos,Total,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,LAT.013,"Burrata ""LS"" GR.125 LOTE. cad. / /21",5,"15,00",IVA 4%,"75,00",31,36,86.111111,0,1,0.0
1,POM.001,"Pomodori Pelati Kg.2,5 ""CORRADO""",6,"3,80",IVA 10%,"22,80",30,32,93.750000,0,1,0.0


In [399]:
code  = this_data.iloc[:, 0]
#codingo = this_data[code]
codingo_data  =  pd.DataFrame(code)
codingo_data


,Código
0,LAT.013
1,POM.001


In [400]:
prec = []
str1 = "Precio"
str2 = "Base I"
str3 = "PVP"
str4 = 'Importe'
str5 = 'Euro Uni.'
str6 = "IMP"
price =''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        prec.append(i)
        price = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        prec.append(i)
        price =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        prec.append(i)
        price =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        prec.append(i)
        price =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        prec.append(i)
        price =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        prec.append(i)
        price =i
if len(prec)>1:
    price = prec[0]
else:
    pass

Precio unit.
Im condition 1
Im condition 6
Impuestos


In [401]:
if price !='':
    price_col = this_data[price]
    price_data = pd.DataFrame(price_col,columns=[price])
    price_data
else:
    print("Price Is Empty")

In [402]:
str1 = "DTO"
str2 = "Descuento"
str3 = "DCTO"
str4 = "Desc"
disc = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        disc = i
        print("Im condition 1")
    
    elif ratio2>=75:
        print(i)
        disc = i
        print("Im condition 2")
    
    elif ratio3>=75:
        print(i)
        disc = i
        print("Im condition 3")
    
    elif ratio4>=75:
        print(i)
        disc = i
        print("Im condition 4")
    
print(disc)

In [403]:
if disc!='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    disc_data

In [404]:
str1 = "Cantidad"
str2 = "Cant"
str3 = "Unid"
str4 = "Unidades"
str5 = "Uds"
str6 = "Peso"
cat = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        cat = i
        print("Im condition 1")
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        cat =i
        print("Im condition 2")
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        cat =i
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        cat =i
    ratio5 = fuzz.ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        cat =i
    ratio6 = fuzz.ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        cat =i

Cantidad
Im condition 1


In [405]:
if cat!='':
    Cantidad = this_data[cat]
    Cantidad_data = pd.DataFrame(Cantidad,columns=[cat])
    Cantidad_data
else:
    print("Quantity is Empty")
#Cantidad_data

In [406]:
str1 = 'IMPUESTO'
imposto = ''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        imposto = i
print(imposto)

Impuestos
Impuestos


In [407]:
if imposto!='':
    imposto_col = this_data[imposto]
    imposto_data = pd.DataFrame(imposto_col,columns=[imposto])
    imposto_data

In [408]:
str1 = 'Tasa'
str2 = 'TIPO'
tasa = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        tasa = i
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        tasa = i
print(tasa)

In [409]:
if tasa!='':
    tasa_col = this_data[tasa]
    tasa_data = pd.DataFrame(tasa_col,columns=[tasa])
    tasa_data

In [410]:
str1 = 'Lote'
str2 = 'LOTE'
lote = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        lote = i
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        lote = i
print(lote)

In [411]:
if lote!='':
    lote_col = this_data[lote]
    lote_data = pd.DataFrame(lote_col,columns=[lote])
    lote_data

In [412]:
lote_data

,Lote
0,22-05-31
1,22-10-31
2,23-09-30
3,23-09-30
4,23-08-31
5,23-04-30
6,29-06-30
7,23-03-01
8,23-09-30
9,22-10-30


In [413]:
#codingo_data
#dataf
#Cantidad_data
#price_data

In [414]:
if disc =='' and imposto =='' and tasa =='' and cat=='' and price == '':
    df2 = pd.concat([codingo_data,desc_data],axis=1, join='inner')
    df2['Price'] = '0'
    df2['Quantity'] = '0'
    df2['Discount'] = '0'
    df2['IMPUESTO']=0
    df2['Tasa']=0
   
   

elif disc!='' and imposto =='' and tasa =='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif disc!='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,imposto_data],axis=1, join='inner')
    df2['Tasa']=0
    
elif disc!='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,tasa_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data,tasa_data],axis=1, join='inner')
    df2['Discount']='0'
    
elif disc=='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,tasa_data],axis=1, join='inner')
    df2['Discount']='0'
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data],axis=1, join='inner')
    df2['Discount']='0'
    df2['Tasa']=0

elif disc=='' and imposto=='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data],axis=1, join='inner')
    df2['Discount']='0'
    df2['IMPUESTO']=0
    df2['Tasa']=0

In [415]:
df2

,Código,Concepto,Cantidad,Precio unit.,Impuestos,Discount,Tasa
0,LAT.013,"Burrata ""LS"" GR.125 LOTE. cad. / /21",5,"15,00",IVA 4%,0,0
1,POM.001,"Pomodori Pelati Kg.2,5 ""CORRADO""",6,"3,80",IVA 10%,0,0


In [62]:
df2 = df2.replace('',0)

In [63]:
df2.columns.values[0] = "Codigo"
df2.columns.values[1] = "Description"
df2.columns.values[2] = "Cantidad"
df2.columns.values[3] = "Precio"

In [64]:
df2.iloc[:,3]= df2.iloc[:,3].str.replace('[A-Z]', '')
df2.iloc[:,3]= df2.iloc[:,3].str.replace('[/\+%&*-]', '')
df2.iloc[:,3]= df2.iloc[:,3].str.replace(',', '.').astype(float)

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  



In [66]:
df2.iloc[:,2]= df2.iloc[:,2].str.replace(',', '.').astype(float)

In [67]:
df2.iloc[:,4]= df2.iloc[:,4].str.replace(',', '.').astype(float)

In [68]:
df2

,Codigo,Description,Cantidad,Precio,Discount,IMPUESTO,Tasa
1,05109668,AGREF CACAH.CRUNCH TIJ.JR C24UX50G,0.0,0.0,0.0,0,0
2,05105266,GREF MRC ORIGINAL RA C24U,0.0,0.0,0.0,0,0
3,05106569,EGREF MRC XL SR/AL C20UX85G,0.0,0.0,0.0,0,0
4,05105267,GREF MRC MIX-5 RA C24U,0.0,0.0,0.0,0,0
5,03104971,GREF SNATTS CHOC C/PVP SR C10U,0.0,0.0,0.0,0,0


In [69]:
df2.dtypes

Codigo          object
Description     object
Cantidad       float64
Precio         float64
Discount       float64
IMPUESTO         int64
Tasa             int64
dtype: object